In [1]:
import socket
import time
import csv
import random

In [2]:
#API_dataset_csv_path = "/home/ubuntu/jupyter/Datasets/goodreads_API_dataset.csv"
#scraped_dataset_csv_path = "/home/ubuntu/jupyter/Datasets/goodreads_scraped_dataset.csv"

API_dataset_tsv_path = "/home/ubuntu/jupyter/Datasets/goodreads_API_dataset.tsv"
scraped_dataset_tsv_path = "/home/ubuntu/jupyter/Datasets/goodreads_scraped_dataset.tsv"

## here i collect all the paths
dataset_paths = [API_dataset_tsv_path, scraped_dataset_tsv_path]
#dataset_paths = [scraped_dataset_tsv_path]

In [3]:
PRINT_FREQUENCY=1000

In [4]:
# this function is responsible to write the dataset on the socket line by line so that spark
# reads it on the other end.
# the line is a touple from a randoom table

def sendData(connection, dataset):

    print("Start sending data")
    count = 0

    # Open file
    with open(dataset, encoding="utf-8", errors="ignore") as file_obj:
        # Create reader object by passing the file
        # object to reader method

        heading=next(file_obj)

        reader_obj = csv.reader(file_obj, delimiter="\t")

        # Iterate over each row in the csv
        # file using reader object
        for line in reader_obj:
            try:
                #print(line)
                line_string = f'{str(line)}\n'
                byte_message = line_string.encode(encoding='utf-8', errors='ignore')
                #print(f'Debug byte_message {byte_message}')
                connection.sendall(byte_message)
                time.sleep(random.uniform(0, 0.5))  # Aggiungi un delay per non sovraccaricare il ricevitore

                if count % PRINT_FREQUENCY == 0:
                  print("Sent:", line_string)
                count += 1
            except Exception as e:
                print("Error sending data:", e)
                break

In [ ]:
host = "localhost"
port = 9999
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((host, port))
s.listen(5)
print("Server is listening on", port)

try:
  connection, addr = s.accept()
  print("Connected by", addr)

  for path in dataset_paths:
    sendData(connection, path)
except Exception as e:
  print("Connection error:", e)
finally:
  connection.close()

Server is listening on 9999
